In [1]:
!pip install dask[complete]
!pip install dask-ml

     ---------------------------------------- 0.0/148.7 kB ? eta -:--:--
     -- ------------------------------------- 10.2/148.7 kB ? eta -:--:--
     ------- ----------------------------- 30.7/148.7 kB 330.3 kB/s eta 0:00:01
     ------------------------------------ - 143.4/148.7 kB 1.2 MB/s eta 0:00:01
     -------------------------------------- 148.7/148.7 kB 1.1 MB/s eta 0:00:00
  Obtaining dependency information for dask-glm>=0.2.0 from https://files.pythonhosted.org/packages/4f/73/ad1a7b3a11fa37db0807a9f521dfaeefedc728a2d8cae3d2b071ad4136f3/dask_glm-0.3.0-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/81.0 kB ? eta -:--:--
     ---------------------------------------- 81.0/81.0 kB ? eta 0:00:00


In [2]:
import dask
import dask.dataframe as dd
import dask_ml

Initializing DASK instance

In [3]:
from dask.distributed import Client

client = Client(n_workers = 6, threads_per_worker = 1)